In [1]:
!pip install beautifulsoup4


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [2]:
!pip install requests


[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [4]:
!pip install pandas

     ---------------------------------------- 10.4/10.4 MB 3.5 MB/s eta 0:00:00
     ---------------------------------------- 14.8/14.8 MB 3.9 MB/s eta 0:00:00
     -------------------------------------- 499.4/499.4 kB 5.2 MB/s eta 0:00:00



[notice] A new release of pip available: 22.3 -> 22.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup as bs

In [59]:
URL_TEMPLATE = "https://www.dotabuff.com/players"
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/108.0.0.0 Safari/537.36'}

r = requests.get(URL_TEMPLATE, headers=headers)
print(r.status_code)

200


In [60]:
soup = bs(r.text, 'html.parser')
players = soup.find_all('td', class_='cell-large')
#print(players)
d = {}
for name in players:
    d[name.a.text] = {'link': name.a['href'], 'syn': name.a.text}
print(d)

{'Dendi': {'link': '/players/70388657', 'syn': 'Dendi'}, 'EG.Arteezy': {'link': '/players/86745912', 'syn': 'EG.Arteezy'}, 'NGX.SumaiL-': {'link': '/players/111620041', 'syn': 'NGX.SumaiL-'}, 'Secret.Puppey': {'link': '/players/87278757', 'syn': 'Secret.Puppey'}, 'TSpirit.Yatoro': {'link': '/players/321580662', 'syn': 'TSpirit.Yatoro'}, 'Liquid.MATUMBAMAN': {'link': '/players/72312627', 'syn': 'Liquid.MATUMBAMAN'}, 'syndereN': {'link': '/players/4281729', 'syn': 'syndereN'}, 'KuroKy': {'link': '/players/82262664', 'syn': 'KuroKy'}, 'Liquid.zai': {'link': '/players/73562326', 'syn': 'Liquid.zai'}, 'Secret.iceiceice': {'link': '/players/84772440', 'syn': 'Secret.iceiceice'}, 'PSG.LGD.Ame': {'link': '/players/898754153', 'syn': 'PSG.LGD.Ame'}, 'ana': {'link': '/players/311360822', 'syn': 'ana'}, 'EG.Cr1t-': {'link': '/players/25907144', 'syn': 'EG.Cr1t-'}, 'Secret.Nisha': {'link': '/players/121769650', 'syn': 'Secret.Nisha'}, 'PSG.LGD.Faith_bian': {'link': '/players/118134220', 'syn': 'PS

In [61]:
player_url = "https://www.dotabuff.com"
ch = str(input())
r_player = requests.get(player_url + d[ch]['link'], headers=headers)
print(r_player.status_code)

Dendi
200


In [2]:
class DotabuffParser():
    
    import pandas as pd
    import requests
    from bs4 import BeautifulSoup as bs
    
    def __init__(self, header):
        self.URL_TEMPLATE = "https://www.dotabuff.com"
        self.headers = {'User-Agent': header}
        self.players_dct = {}
        self.url = '/players'
        self.con = self.connect_to(self.url)
        if self.con.status_code == 200:
            self.soup = bs(self.con.text, 'html.parser')
            self.players = self.soup.find_all('td', class_='cell-large')
            for name in self.players:
                self.players_dct[name.a.text] = {'link': name.a['href'], 'syn': name.a.text}
        else:
            print(f'Ошибка подключения (Код: {self.con})')
            del()
    
    def connect_to(self, url = ''):
        self.con = requests.get(self.URL_TEMPLATE + url, headers = self.headers)
        return self.con   
           
    def get_player(self, name):
        self.name = name
        self.action = {"1": self.get_best_heros(self.name), "2": self.get_matches(self.name)
            
        }
        while True: 
            self.chose = str(input("\n1 - показать популярных героев\n2 - показать последние матчи\ne - выход\n"))
            if self.chose != 'e':
                print()
                self.get_player_header(self.name)
                self.action[self.chose]
            else:
                break
    
    def get_top_players(self):
        return self.players_dct
 
    def get_player_header(self, name):
        self.name = name
        self.con = self.connect_to(self.players_dct[self.name]['link'])
        if self.con.status_code == 200:
            self.soup = bs(self.con.text, 'html.parser')
            self.header = self.soup.find('div', class_ = 'header-content-secondary')
            self.last_game = self.header.time['datetime'].split('+')[0]
            self.last_game = self.last_game.split('T')
            self.match_scores = [int(self.header.find('span', class_ = 'wins').text.replace(',', '')),
                                 int(self.header.find('span', class_ = 'losses').text.replace(',', '')),
                                 int(self.header.find('span', class_ = 'abandons').text.replace(',', ''))]
            self.win_pr = self.match_scores[0] / sum(self.match_scores) * 100
            print(f'Игрок: {self.name}'.ljust(59))
            print(f'Последняя игра: {self.last_game[0]} в {self.last_game[1]}')
            print(f'Матчи: {self.match_scores[0]} — {self.match_scores[1]} — {self.match_scores[2]}')
            print(f'Доля побед: {round(self.win_pr, 2)}%')
        else:
            return f'Ошибка подключения (Код: {self.con})', 'get_player_header'
    
    def get_matches(self, name):
        self.name = name
        self.matches = {}
        self.wl = {'Won Match': 'Победа', 'Lost Match': 'Поражение'}
        self.con = self.connect_to(self.players_dct[self.name]['link'])
        if self.con.status_code == 200:
            self.soup = bs(self.con.text, 'html.parser')
            self.table_matches = self.soup.find('div', class_='r-table r-only-mobile-5 performances-overview')
            self.all_matches = self.table_matches.find_all('div', class_ = 'r-row')

            for name in self.all_matches:
                self.matches[name.a.img['title']] = [
                    name.find('div', class_ = 'r-fluid r-175 r-text-only r-right r-match-result').a.text,
                    name.find('div', class_ = 'subtext').text]
            print(' Последние игры '.center(59, '—'))
            print("\033[4m{}".format('Герой'.center(25)+'Ранг'.center(18)+'Результат'.center(16)))
            self.i = 1
            for k, item in self.matches.items():
                a, b, c = (str(self.i) + ') ' + k).ljust(25), self.wl[item[0]].ljust(11), item[1].center(16)
                print(f'{a}||{c}|| {b}||')
                self.i += 1
            #print('—' * 59)
        else:
            return f'Ошибка подключения (Код: {self.con})', 'get_matches'
    
    def get_best_heros(self, name):
        self.name = name
        self.matches = {}
        self.con = self.connect_to(self.players_dct[self.name]['link'])
        if self.con.status_code == 200:
            self.soup = bs(self.con.text, 'html.parser')
            self.table_top = self.soup.find('div', class_='r-table r-only-mobile-5 heroes-overview')
            self.top_matches = self.table_top.find_all('div', class_ = 'r-row')
            for name in self.top_matches:
                k = name.find_all('div', class_ = 'r-fluid r-10 r-line-graph')
                self.matches[name.a.img['title']] = [int(k[0].find('div', class_ = 'r-body').text.replace(',', '')),
                                                     k[1].find('div', class_ = 'r-body').text,
                                                    name.find('span', class_ = 'primary-lane-text').text]
            print(' Лучшие показатели '.center(60, '—'))
            print("\033[4m{}".format('Герой'.center(28)+'Игр'.center(10)+'Винрейт'.center(8)+'Позиция'.center(14)))
            for k, item in self.matches.items():
                a, b, c, d = k.ljust(25), str(item[0]).center(7), item[1].rjust(7), item[2].rjust(11)
                print(f'||{a}||{b}||{c}||{d}||')
        else:
            return f'Ошибка подключения (Код: {self.con})', 'get_best_heros'


In [ ]:
dbp = DotabuffParser('Mozilla/5.0 (X11; Ubuntu; Linux i686; rv:106.0) Gecko/20100101 Firefox/106.0')

dbp.get_player('Beastcoast.K1 Hector')

#dbp.get_matches('TSpirit.Yatoro')
#dbp.get_best_heros('TSpirit.Yatoro')

———————————————————— Лучшие показатели —————————————————————
           Герой               Игр    Винрейт    Позиция    
||Huskar                   ||  434  || 59.91%|| Safe Lane ||
||Troll Warlord            ||  414  || 59.18%|| Safe Lane ||
||Juggernaut               ||  407  || 51.35%|| Safe Lane ||
||Anti-Mage                ||  400  || 53.75%|| Safe Lane ||
||Terrorblade              ||  341  || 57.77%|| Safe Lane ||
||Bloodseeker              ||  226  || 62.39%|| Safe Lane ||
||Alchemist                ||  222  || 56.31%|| Safe Lane ||
||Phantom Assassin         ||  200  || 59.00%|| Safe Lane ||
||Slark                    ||  183  || 50.27%|| Safe Lane ||
||Spectre                  ||  168  || 54.76%|| Safe Lane ||
—————————————————————— Последние игры —————————————————————
          Герой                 Ранг          Результат    
1) Wraith King           ||    Immortal    || Победа     ||
2) Troll Warlord         ||    Immortal    || Победа     ||
3) Chaos Knight          || 

In [3]:
dbp.get_player_header('TSpirit.Yatoro')

NameError: name 'dbp' is not defined

In [181]:
dbp.get_top_players()

{'Dendi': {'link': '/players/70388657', 'syn': 'Dendi'},
 'EG.Arteezy': {'link': '/players/86745912', 'syn': 'EG.Arteezy'},
 'NGX.SumaiL-': {'link': '/players/111620041', 'syn': 'NGX.SumaiL-'},
 'Secret.Puppey': {'link': '/players/87278757', 'syn': 'Secret.Puppey'},
 'TSpirit.Yatoro': {'link': '/players/321580662', 'syn': 'TSpirit.Yatoro'},
 'Liquid.MATUMBAMAN': {'link': '/players/72312627',
  'syn': 'Liquid.MATUMBAMAN'},
 'syndereN': {'link': '/players/4281729', 'syn': 'syndereN'},
 'KuroKy': {'link': '/players/82262664', 'syn': 'KuroKy'},
 'Liquid.zai': {'link': '/players/73562326', 'syn': 'Liquid.zai'},
 'Secret.iceiceice': {'link': '/players/84772440', 'syn': 'Secret.iceiceice'},
 'PSG.LGD.Ame': {'link': '/players/898754153', 'syn': 'PSG.LGD.Ame'},
 'ana': {'link': '/players/311360822', 'syn': 'ana'},
 'EG.Cr1t-': {'link': '/players/25907144', 'syn': 'EG.Cr1t-'},
 'Secret.Nisha': {'link': '/players/121769650', 'syn': 'Secret.Nisha'},
 'PSG.LGD.Faith_bian': {'link': '/players/118134